In [ ]:
import pandas as pd
import numpy as np

In [ ]:
restaurants_df = pd.read_csv('restaurants.csv')
restaurants_df.head()

In [ ]:
restaurants_df.describe(include='all')

In [ ]:
restaurants_df.shape

In [ ]:
restaurants_df.nunique()

In [ ]:
restaurants_df['text']
               

In [ ]:
restaurants_df[restaurants_df['text'] == 'i love this place']

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)

In [ ]:
tokenizer.fit_on_texts(restaurants_df['text']) 
sequences = tokenizer.texts_to_sequences(restaurants_df['text'])

In [ ]:
sequences

In [ ]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [ ]:
vec_sequences = vectorize(sequences)

In [ ]:
vec_sequences

In [ ]:
labels = restaurants_df.values[:,1:6].astype('float32')

In [ ]:
labels

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()

model.add(Dense(128, activation='relu', input_shape=(10000,)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(vec_sequences, labels, test_size=800)
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=1700)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', patience = 3, verbose=1)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), callbacks = [es])

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
loss_values = history.history['loss']
val_loss_values = history.history['val_loss']
epochs = range(1, (len(history.history['loss']) + 1))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc_values = history.history['accuracy']
val_acc_values = history.history['val_accuracy']
epochs = range(1, (len(history.history['accuracy']) + 1))
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
results = model.evaluate(X_test, y_test)
print("_"*100)
print("Test Loss and Accuracy")
print("results ", results)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
opiniones = ['service', 'food', 'anecdotes', 'price', 'ambience']

In [ ]:
for i in range(5):
    print(predictions[i])

In [ ]:
sorted_categories = predictions[0].argsort()
sorted_categories

In [ ]:
for op in range(12):
    sorted_categories = predictions[op].argsort()
    if sorted_categories[0] >= 0.1: # al menos cree en un 10% en la mejor predicción

        for i in range(4,-1,-1):
            print("{}".format(opiniones[sorted_categories[i]])+" ({:.3})".format(predictions[op][sorted_categories[i]]))
        
        [print("labels: {}".format(opiniones[i])) for i in range(5) if y_test[op][i]]
        print("______________________________________________________________________________________________________")
